# 17. Future를 이용한 동시성

## 17.1. 세가지 스타일의 웹 내려받기

긴 지연시간 동안 CPU 클록을 낭비하지 않기 위해 네트워크 입출력을 효율적으로 처리하려면 동시성을 이용해야 한다. 네트워크에서 응답이 오는 동안 다른 일을 처리하는 것이 좋다.

1. 이미지 내려받아 디스크에 저장 후 다음 이미지 내려 받기
2. 동시에 내려 받기, 도착하느 대로 파일에 저장
  -concurrent.futures 패키지/ asyncio
  
순차적 스크립트보다 동시성 스크립트가 5배 빠름.

### 17.1.1. 순차 내려받기 스크립트
### 17.1.2. concurrent.futures로 내려받기
ThreadPoolExcutor, ProcessPoolExecutor 클래스 : 콜러블 객체를 서로 다른 스레드나 프로세스에서 실행할 수 있게 해주는 인터페이스를 구현
### 17.1.3. Future은 어디?
future : concurrent.futures, asyncio 내부의 핵심 컴포넌트, 사용자에게 드러나지 않는 경우 있음. 사람이 직접 객체 생성하면 안 됨. 대기 중인 작업 큐에 넣고 완료 상태 조사하고 결과를 가져올 수 있도록 캡슐화함. 

## 17.2. 블로킹 I/O와 GIL

GIL : 전역 인터프리터 락.  한번에 한 스레드만 파이썬 바이트코드를 실행하도록 제한해 단일 파이썬 프로세스가 동시에 다중 CPU 코어 사용할 수 없게 함.

블로킹 입출력 시행하는 모든 표준 라이브러리 함수 : OS에서 결과 기다리는 동안 GIL 해제 함.

## 17.3. concurrent.futures로 프로세스 실행

ProcessPoolExecutor 클래스 사용해 작업을 여러 파이썬 프로세스에 분산시켜 진정한 병렬 컴퓨팅을 가능하게 함.

계산 프로그램>> 성능 테스트 >> 최적의 스레드 수 찾아야

arcfour_futures.py : 순수하게 파이썬으로 구현된 RC4 알고리즘을 이용해 149KB ~ 384KB 정도 되는 바이트 배열 10개를 암호화하고 복호화함.
sha_futures.py : OpenSSL 라이브러리를 사용하는 표준 라이브러리 패키지 hashlib를 이용해 1MB 바이트 배열 10개에 대한 SHA-256 해시를 계산한다.

executor.map() : 사용하기 쉽지만 호출한 순서 그대로 결과를 반환함

## 17.5. 진행 상황 출력하고 에러를 처리하며 내려받기

flags2 예제들 >> 다양한 에러 조건의 처리 테스트

flags2 예제 : TQDM 패키지 이용 >> 텍스트 기반의 진행 막대를 애니메이트함.

-s/--server 옵션 : 테스트에 사용한 HTTP 서버와 기반 URL을 지정함. 4개의 문자열 중 하나 전달>>어디서 자료 가져올지 결정함.
          *4가지 문자열 : LOCAL, REMOTE, DELAY, ERROR*

### 17.5.1. flags2 예제에서 에러 처리

download_one() : 404 error( not found) 처리
download_many() : 나머지 에러 처리

### 17.5.2. futures.as_completed() 사용

flags2_threadpool.py

### 17.5.3. 스레드 및 멀티프로세스의 대안

concurrent.futures : 스레드를 사용하기 위해 나중에 추가된 방법

Thread, Lock, Semaphore 등 모듈의 기본 컴포넌트를 이요해 처리 가능
스레드 간 데이터 전송 : queue 모듈이 제공하는 t스레드 안전한 큐 사용 가능

multiprocessing 패키지 : 계산 위주 작업 수행. 데이터 공유 쉽게 해결. >> futures.ProcessPoolExecutor 안 될 떄 사용

# 18. asyncio를 이용한 동시성

## 18.1. 스레드와 코루틴 비교

스레드 ; 파이썬에는 스레드 종료하는 API 정의되어 있지 않음. 스레드에 메시지 보내 종료 시켜야(signal.go 속성 False로 설정>>spinner 스레드가 확인 후 종료시킴)

@asyncio.coroutine : 코루틴을 일반 함수와 다르게 보이도록 만들고, yield from 되지 않고 가비지 컬렉트 되는 경우 경고 메시지 출력, 데커레이트된 제너레이터를 자동으로 기동하지 않음.

supervisor() 함수 차이 
  asyncio.Task는 threading.Thread와 거의 대등함.
  Task는 코루틴을 구동하고, Tread는 콜러블을 호출
  Task 객체는 직접 생성x, 코루틴을 asyncio.async()나 loop.create_task()에 전달해 가져옴
  Task 객체는 이미 asyncio.async() 등에 의해 실행이 스케줄링되어 있음. Thread 객체는 start() 메서드를 호출해 실행하라 명령해야 함.
  스레드화된 supervisor()에서 slow_function()은 평범한 함수로, 스레드에 의해 직접 호출됨. 비동기 supervisor()에서 slow_function()은 yield from으로 구동하는 코루틴임.
  스레드는 외부에서 API를 이용해 중단시킬 수 없음. Task에는 코루틴 안에서 CancelledError를 발생시키는 Task.concel() 객체 메서드가 있음. 코루틴은 중단되었던 yield문에서 예외를 잡아 처리할 수 있음.
  supervisor() 코루틴은 main() 함수 안에서 loop.run_until_complete()로 실행해야함.

스레드 : 스케줄러가 언제든 스레드를 중단 시킬 수 있음. >> 크리티컬 섹션 보호 위해 락을 잠그고 작업 수행 중 인터럽트되지 않게 해야 함. 

코루틴 :모든 것이 기본적으로 인터럽트로부터 보호됨. 명시적으로 yield 실행해야 다른 부분 실행 됨.

스레드 연산 동기화 위해 코루틴 중 하나만 사용하면 됨. yield나 yield from을 이용해 제어권을 스케줄러에 넘겨줄 수 있다. 따라서 코루틴은 안전하게 취소 가능.

### 18.1.1. asyncio.Future : 논블로킹 설계

언젠가 concurren.futures.Future에 __iter__() 메서드를 추가해 yield from에 사용하게 하는 등 asyncio.Future와 concurren.futures.Future를 통합할 수도 있다.

Future : 실행할 코드를 스케줄링해야 생성됨. 

asyncio 의 BaseEventLoop.create_task() 메서드 : 코루틴을 받아 실행하기 위해 스케줄링하고 asyncio.Task 객체를 반환함. Task 객체는 Future 객체이기도 함.

done(), add_done_callback(), result()등의 메서드 제공

result() : 인수 받지 않음 >> 시간초과 지정x . 실행 완료 안된 Future 객체의 result() 호출하면 asyncio.InvalidStateError 예외가 발생 >> 결과는 yield from 이용해 가져옴

### 18.1.2. Future, Task, 코루틴에서 생성하기

res = yield from foo() : foo()가 코루틴 객체나 Future나 Task 객체 반환하는 일반함수면 코드 작동함.

코루틴 받아 Task 객체 가져오는 방법
    asyncio.async(coro_or_future, * , loop=None) 
    BaseEventLoop.create_task(coro)
    
## 18.2. asyncio와 aiohttp로 내려받기

asyncio : TCP, UDP만 직접 지원. 다른 프로토콜 지원하려면 서드파티 패키지 필요

yield from
  yield from으로 연결된 전체 코루틴 체인은 궁극적으로 가장 바깥쪽에 있는 대표 제너레이터의 next()나 send()를 명시적/암묵적으로 호출하는 비코루틴 호출자에 의해 구동됨
  이 체인 가장 안쪽에 있는 하위 제너레이터는 단지 yield를 사용하는 단순 제너레이터이거나 반복형 객체여야 한다.
  
ascyncio API와 yield from을 사용할 때
  우리가 만든 코루틴 체인은 언제나 가장 바깥쪽 대표 제너레이터를 loop.run_until_complet() 등의 asyncio API에 전달함으로써 구동된다. 즉 asyncio를 사용할 때는 next()나 send() 메서드를 직접 호출해 코루틴 체인을 구동하는 것이 아니라, asyncio 이벤트 루프가 처리하도록 해야 한다.
  우리가 만든 코루틴 체인은 언제나 어떤 asyncio 코루틴 함수나 코루틴 메서드, 혹은 상위 수준 프로토콜을 구현하는 라이브러리의 코루틴에 yield from을 호출하면서 끝나야 한다. 즉, 가장 안쪽의 하위제너레이터는 우리가 만든 코드가 아니라, 실제로 입출력을 수행하는 라이브러리 함수여야 한다.
  
## 18.3. 블로킹 호출을 에둘러 실행하기

블로킹 함수가 전체 애플리케이션의 실행을 멈추지 않게 하는 방법
 -블로킹 연산을 각기 별도의 스레드에서 실행
 -모든 블로킹 연산을 논블로킹 비동기 연산으로 바꾼다.

콜백>> 비동기 호출 >> 논블로킹으로 만들 수 있음.

## 18.4. asyncio 내려받기 스크립트 개선

### 18.4.1. asyncio.as_completed() 사용하기

원래 : 코루틴 리스트 asyncio.wait()에 전달 >> loop.run_until.complete()에 구동되는 await()가 내려받는 작업 모두 완료된 후 결과 반환

>>

### 18.4.2. Executor를 이용해 이벤트 루프 블로킹 피하기

## 18.5. 콜백에서 Future와 코루틴으로

콜백 지옥,,  : 어떤 연산이 다른 연산에 종속적일 때 콜백 안에 콜백이 들어가는 것

세 개의 비동기 작업 연속으로 처리 : 코루틴 체인에서 yield 3번 사용해 이벤트 루프가 계속 실행할 수 있게 해주면 됨. 결과 나오면 코루틴 send() 호출해 코루틴 활성화하면 됨. 

### 18.5.1. 한 번 내려받을 때 여러가지 요청하기

flags2_asyncio.py에서 수정된 코드
  download_one() : yield from을 사용해 get_flag() 및 새로 만들어진 get_country() 코루틴에 위입
  get_flag() : 이 코루틴에 들어 있던 코드 대부분은 get_country()에서도 사용할 수 있도록 새로 만들어진 http_get() 코루틴으로 이동됨.
  get_country() : 이 코루틴은 국가 코드에 대한 metadata.json 파일을 가져와 국가명을 꺼낸다.
  http_get() : 웹에서 파일을 가져오기 위해 공통으로 사용하는 코드

## 18.6. asyncio 서버 작성

### 18.6.1. asyncio TCP 서버
### 18.6.2. aiohttp 웹 서버
### 18.6.3. 동시성을 향상시키는 똑똑한 클라이언트

